# Set-up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# SET TO PROJECT FOLDER
dir = '/content/drive/MyDrive'

# LOCATION OF FEATURE FOLDER
folder = '/wav2vec2_features'

# Get Wav2Vec features from .wav files


Tutorial: [Speech Recognition with Wav2Vec ](https://pytorch.org/audio/stable/tutorials/speech_recognition_pipeline_tutorial.html)

In [ ]:
# ! unzip '/content/drive/MyDrive/cse256/wav2vec2_features.zip'

In [ ]:
import IPython
import matplotlib.pyplot as plt
# from torchaudio.utils import download_asset

In [4]:
import glob
# speech_files = glob.glob("output/*.wav")

In [ ]:
# import torch
# import torchaudio

# print(torch.__version__)
# print(torchaudio.__version__)

# torch.random.manual_seed(0)
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# print(device)

In [ ]:
# bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

# print("Sample Rate:", bundle.sample_rate)

# print("Labels:", bundle.get_labels())

# model = bundle.get_model().to(device)

# print(model.__class__)

In [ ]:
# def file_to_features(file_path):
#   waveform, sample_rate = torchaudio.load(file_path)
#   waveform = waveform.to(device)

#   if sample_rate != bundle.sample_rate:
#     waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

#   with torch.inference_mode():
#     features, _ = model.extract_features(waveform)

#   return features

In [ ]:
import pickle
import os
import numpy as np

# GET FEATURES FROM WAV FILES

# for i, f in enumerate(speech_files):
#   name = f.split(".")[0].split('/')[1]
#   file_name = dir + '/wav2vec2_features/' + name + '.pickle'
#   print('Audio ', i, ': ', file_name)

#   if not os.path.exists(file_name):
#     feat = file_to_features(f)

#     feat = [t.reshape((t.shape[1], t.shape[2])) for t in feat]
#     stacked_feat = torch.stack(feat)
#     features_np = stacked_feat.numpy()

#     average_features = np.mean( np.array(np.split(features_np, 12, axis=0)), axis=0)
#     average_features = average_features.reshape((average_features.shape[1], average_features.shape[2]))

#     with open(file_name, 'wb') as handle:
#       pickle.dump(average_features, handle)

# Features DF

In [ ]:
# import pandas as pd

# meta = pd.read_csv('drive/MyDrive/cse256/speakers_all.csv')
# display(meta)

# Load in audio features

In [ ]:
wav2vec_list = glob.glob(dir + folder + "/*.pickle")

# PCA Clustering

In [7]:
# LOAD IN WAV2VEC FEATURE EMBEDDINGS

import pickle

lang_list = ['english', 'spanish', 'arabic', 'mandarin', 'french', 'korean']

speaker_to_embeddings = {}
speaker_labels = {}
speaker_to_count = {}

for lang in lang_list:
  speaker_to_count[lang] = 0


for lang in lang_list:
  for f in wav2vec_list[:]:
    name = f.split('/')[-1].split('.')[0]

    if (lang in name) and (speaker_to_count[lang] <= 165):
      speaker_features = pickle.load(open(f, 'rb'))
      if speaker_features.shape[0] <= 2000:
        speaker_to_embeddings[name] = speaker_features
        speaker_labels[name] = lang
        speaker_to_count[lang] += 1
        print(lang)

In [ ]:
# PAD FEATURES TO MAKE SAME LENGTH

import numpy as np

min = 5000000000000
max = 0
sum = 0
count = 0
for s in speaker_to_embeddings:
  sum += speaker_to_embeddings[s].shape[0]
  count += 1
  if speaker_to_embeddings[s].shape[0] > max:
    max = speaker_to_embeddings[s].shape[0]
  if speaker_to_embeddings[s].shape[0] < min:
    min = speaker_to_embeddings[s].shape[0]

average = sum/count

print("average embedding length: ", average)
print("Max embedding length: ", max)
print("Min embedding length: ", min)

# average = 1000
num_greater = 0
lengths = []
for s in speaker_to_embeddings:
  if speaker_to_embeddings[s].shape[0] > average:
    lengths.append(speaker_to_embeddings[s].shape[0])
    num_greater += 1

print("Num greater than average: ", num_greater)
print("List of greater: ", lengths)
print("Total speakers: ", count)

average embedding length:  1334.2740471869329
Max embedding length:  1999
Min embedding length:  822
Num greater than average:  234
List of greater:  [1555, 1574, 1837, 1706, 1340, 1606, 1345, 1587, 1363, 1610, 1437, 1449, 1410, 1866, 1431, 1763, 1701, 1338, 1460, 1366, 1352, 1499, 1479, 1762, 1537, 1436, 1833, 1456, 1399, 1339, 1536, 1563, 1347, 1524, 1706, 1559, 1472, 1467, 1896, 1819, 1967, 1934, 1940, 1828, 1342, 1633, 1537, 1627, 1857, 1649, 1535, 1660, 1503, 1360, 1851, 1829, 1572, 1782, 1831, 1678, 1784, 1804, 1529, 1511, 1697, 1365, 1551, 1499, 1523, 1704, 1682, 1371, 1401, 1769, 1584, 1682, 1721, 1999, 1528, 1799, 1346, 1447, 1754, 1431, 1923, 1599, 1678, 1937, 1804, 1427, 1791, 1429, 1749, 1363, 1625, 1511, 1924, 1386, 1781, 1983, 1416, 1631, 1959, 1512, 1752, 1620, 1497, 1493, 1402, 1867, 1754, 1915, 1417, 1436, 1994, 1888, 1554, 1539, 1829, 1955, 1416, 1971, 1365, 1827, 1858, 1663, 1592, 1888, 1604, 1492, 1913, 1453, 1626, 1598, 1562, 1503, 1500, 1337, 1570, 1570, 1734, 175

In [ ]:

speaker_to_padded = {}
for s in speaker_to_embeddings:
  if speaker_to_embeddings[s].shape[0] > average:
    embed_copy = np.copy(speaker_to_embeddings[s])
    right_length = embed_copy[:int(average),:]
  elif speaker_to_embeddings[s].shape[0] < average:
    diff = int(average) - speaker_to_embeddings[s].shape[0]
    right_length =  np.pad(speaker_to_embeddings[s], ((diff,0), (0, 0)), 'constant')

  flattened = right_length.flatten()
  speaker_to_padded[s] = flattened

print("Padded shape: ", flattened.shape)

Padded shape:  (1024512,)


In [ ]:
from sklearn.decomposition import PCA

values = list(speaker_to_padded.values())
names = list(speaker_to_padded.keys())
langs = [speaker_labels[n] for n in names]

pca = PCA(n_components=3)
components_pca = pca.fit_transform(values)

In [ ]:
pca.explained_variance_ratio_

array([0.04095181, 0.01697611, 0.0145474 ])

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_3d(components_pca, x=0, y=1, z=2, color=langs)
fig.show()

In [ ]:
fig = px.scatter_matrix(
    components_pca,
    labels=langs,
    dimensions=range(3),
    color=langs
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
lang_to_comps = {}
for lang in lang_list:

  if lang not in lang_to_comps:
    lang_to_comps[lang] = []

  for name, comp in zip(names, components_pca):
    if lang in name:
      lang_to_comps[lang].append(comp)


lang_to_centroid = {}
for lang in lang_to_comps:
  centroid = np.average(np.asarray(lang_to_comps[lang]), axis=0)
  lang_to_centroid[lang] = centroid


In [ ]:
for lang in lang_to_comps:
  comps = lang_to_comps[lang]
  variance = np.var(comps)
  print(lang, ' cluster variance: ', variance)

english  cluster variance:  934.4075217927992
spanish  cluster variance:  778.7761945008389
arabic  cluster variance:  565.1353076098462
mandarin  cluster variance:  744.475575560465
french  cluster variance:  795.6967262026521
korean  cluster variance:  642.7713204853114


In [ ]:
for lang1 in lang_to_centroid:
  if lang1 == 'english':
    for lang2 in lang_to_centroid:
      if lang1!=lang2:
        distance = np.linalg.norm(lang_to_centroid[lang1] - lang_to_centroid[lang2])
        print(lang1, '-', lang2, ': ', distance)
    print()

english - spanish :  29.042886442021018
english - arabic :  45.3913399764493
english - mandarin :  37.3744167820694
english - french :  27.900726442641044
english - korean :  34.87422811465723



# UMAP Clustering

In [ ]:
!pip3 uninstall umap
!pip3 install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=6a11841f6502f0fe43b4530167731b0eef247671e668cda899a0ef2f08992fb3
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=70282f97bccb171c9da8e3ad0793d8e9919f63b685b9dd0a23115b78d6b28d1e
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
import umap.umap_ as UMAP

umap = UMAP.UMAP(n_components=3, init='random', random_state=0)
components_umap = umap.fit_transform(values)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.cross_decomposition import PLSRegression
import pandas as pd

# UMAP Component #1

# UMAP_matrix = pd.DataFrame(components_umap[:, 0:1])
# pls = PLSRegression(n_components = 1)
# pls.fit(UMAP_matrix, values)
# y_pred = pls.predict(UMAP_matrix)
# print(r2_score(values, y_pred, multioutput = 'variance_weighted'))

In [ ]:
# UMAP Component #2

# UMAP_matrix = pd.DataFrame(components_umap[:, 1:2])
# pls = PLSRegression(n_components = 1)
# pls.fit(UMAP_matrix, values)
# y_pred = pls.predict(UMAP_matrix)
# print(r2_score(values, y_pred, multioutput = 'variance_weighted'))

In [ ]:
# UMAP Component #3

# UMAP_matrix = pd.DataFrame(components_umap[:, 2:3])
# pls = PLSRegression(n_components = 1)
# pls.fit(UMAP_matrix, values)
# y_pred = pls.predict(UMAP_matrix)
# print(r2_score(values, y_pred, multioutput = 'variance_weighted'))

In [ ]:
fig = px.scatter_3d(components_pca, x=0, y=1, z=2, color=langs)
fig.show()

In [ ]:
fig = px.scatter_matrix(
    components_umap,
    labels=langs,
    dimensions=range(3),
    color=langs
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
lang_to_comps = {}
for lang in lang_list:

  if lang not in lang_to_comps:
    lang_to_comps[lang] = []

  for name, comp in zip(names, components_umap):
    if lang in name:
      lang_to_comps[lang].append(comp)


lang_to_centroid = {}
for lang in lang_to_comps:
  centroid = np.average(np.asarray(lang_to_comps[lang]), axis=0)
  lang_to_centroid[lang] = centroid


In [ ]:
for lang in lang_to_comps:
  comps = lang_to_comps[lang]
  variance = np.var(comps)
  print(lang, ' cluster variance: ', variance)

english  cluster variance:  3.4213488
spanish  cluster variance:  5.4361396
arabic  cluster variance:  6.1959705
mandarin  cluster variance:  6.245115
french  cluster variance:  4.3873525
korean  cluster variance:  5.5276036


In [ ]:
for lang1 in lang_to_centroid:
  if lang1 == 'english':
    for lang2 in lang_to_centroid:
      if lang1!=lang2:
        distance = np.linalg.norm(lang_to_centroid[lang1] - lang_to_centroid[lang2])
        print(lang1, '-', lang2, ': ', distance)
    print()

english - spanish :  1.7449203
english - arabic :  2.9911125
english - mandarin :  2.6417236
english - french :  1.4637507
english - korean :  2.265379



# t-SNE Clustering

In [ ]:
from sklearn.manifold import TSNE

values = np.asarray(values)
tsne = TSNE(n_components=3)
components_tsne = tsne.fit_transform(values)

In [ ]:
# tSNE Component #1
# tSNE_matrix = pd.DataFrame(components_tsne[:, 0:1])
# pls = PLSRegression(n_components = 1)
# pls.fit(tSNE_matrix, values)
# y_pred = pls.predict(tSNE_matrix)
# print(r2_score(values, y_pred, multioutput = 'variance_weighted'))

In [ ]:
# tSNE Component #2
# tSNE_matrix = pd.DataFrame(components_tsne[:, 1:2])
# pls = PLSRegression(n_components = 1)
# pls.fit(tSNE_matrix, values)
# y_pred = pls.predict(tSNE_matrix)
# print(r2_score(values, y_pred, multioutput = 'variance_weighted'))

In [ ]:
# tSNE Component #3
# tSNE_matrix = pd.DataFrame(components_tsne[:, 2:3])
# pls = PLSRegression(n_components = 1)
# pls.fit(tSNE_matrix, values)
# y_pred = pls.predict(tSNE_matrix)
# print(r2_score(values, y_pred, multioutput = 'variance_weighted'))

In [ ]:
fig = px.scatter_3d(components_tsne, x=0, y=1, z=2, color=langs)
fig.show()

In [ ]:
fig = px.scatter_matrix(
    components_tsne,
    labels=langs,
    dimensions=range(3),
    color=langs
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
lang_to_comps = {}
for lang in lang_list:

  if lang not in lang_to_comps:
    lang_to_comps[lang] = []

  for name, comp in zip(names, components_tsne):
    if lang in name:
      lang_to_comps[lang].append(comp)


lang_to_centroid = {}
for lang in lang_to_comps:
  centroid = np.average(np.asarray(lang_to_comps[lang]), axis=0)
  lang_to_centroid[lang] = centroid


In [ ]:
for lang in lang_to_comps:
  comps = lang_to_comps[lang]
  variance = np.var(comps)
  print(lang, ' cluster variance: ', variance)

In [ ]:
for lang1 in lang_to_centroid:
  if lang1 == 'english':
    for lang2 in lang_to_centroid:
      if lang1!=lang2:
        distance = np.linalg.norm(lang_to_centroid[lang1] - lang_to_centroid[lang2])
        print(lang1, '-', lang2, ': ', distance)
    print()